In [7]:
from rouge import Rouge
import glob
import os
import re
import pandas as pd

#goldSummaries = glob.glob(r'..data/raw/OpinosisDataset1.0_0/summaries-gold/*/')
gold_summaries = os.walk(r'../data/raw/OpinosisDataset1.0_0/summaries-gold')
gold_list = next(gold_summaries)[1]

In [70]:
summaryFiles = glob.glob(r'../data/processed/textrank/*')
for h in summaryFiles:
    foldername_search = re.search(r'[^\\/:*?"<>|\r\n]+$', h)
    foldername = (foldername_search.group()).split('.')[0]
    with open(h, 'r') as f:
        hypothesis = f.read()
    f.close()
    if foldername in gold_list:
        files = glob.glob('../data/raw/OpinosisDataset1.0_0/summaries-gold/'+foldername+'/*')
        for r in files:
            with open(r, 'r') as f:
                reference = f.read()
                rouge = Rouge()
                scores = rouge.get_scores(hypothesis, reference)[0]
                rouge_1.append(pd.DataFrame(scores['rouge-1'], index=[foldername]))
        frame = pd.concat(rouge_1)
frame

,f,p,r
accuracy_garmin_nuvi_255W_gps,0.152381,0.096386,0.363636
accuracy_garmin_nuvi_255W_gps,0.122449,0.072289,0.400000
accuracy_garmin_nuvi_255W_gps,0.108696,0.060241,0.555556
accuracy_garmin_nuvi_255W_gps,0.164948,0.096386,0.571429
accuracy_garmin_nuvi_255W_gps,0.104167,0.060241,0.384615
bathroom_bestwestern_hotel_sfo,0.138614,0.090909,0.291667
bathroom_bestwestern_hotel_sfo,0.065934,0.038961,0.214286
bathroom_bestwestern_hotel_sfo,0.129032,0.077922,0.375000
bathroom_bestwestern_hotel_sfo,0.081633,0.051948,0.190476
bathroom_bestwestern_hotel_sfo,0.086022,0.051948,0.250000


In [71]:
summaryFiles = glob.glob(r'../data/processed/lexrank/*')
dictionary_scores = dict()
for h in summaryFiles:
    foldername_search = re.search(r'[^\\/:*?"<>|\r\n]+$', h)
    foldername = (foldername_search.group()).split('.')[0]
    with open(h, 'r') as f:
        hypothesis = f.read()
    f.close()
    if foldername in gold_list:
        files = glob.glob('../data/raw/OpinosisDataset1.0_0/summaries-gold/'+foldername+'/*')
        for r in files:
            with open(r, 'r') as f:
                reference = f.read()
                rouge = Rouge()
                scores = rouge.get_scores(hypothesis, reference)[0]
                rouge_1.append(pd.DataFrame(scores['rouge-1'], index=[foldername]))
        frame = pd.concat(rouge_1)
frame

,f,p,r
accuracy_garmin_nuvi_255W_gps,0.152381,0.096386,0.363636
accuracy_garmin_nuvi_255W_gps,0.122449,0.072289,0.400000
accuracy_garmin_nuvi_255W_gps,0.108696,0.060241,0.555556
accuracy_garmin_nuvi_255W_gps,0.164948,0.096386,0.571429
accuracy_garmin_nuvi_255W_gps,0.104167,0.060241,0.384615
bathroom_bestwestern_hotel_sfo,0.138614,0.090909,0.291667
bathroom_bestwestern_hotel_sfo,0.065934,0.038961,0.214286
bathroom_bestwestern_hotel_sfo,0.129032,0.077922,0.375000
bathroom_bestwestern_hotel_sfo,0.081633,0.051948,0.190476
bathroom_bestwestern_hotel_sfo,0.086022,0.051948,0.250000


In [ ]:
summaryFiles = glob.glob(r'../data/processed/textrank_cosine/*')
fileNames = []

for h in summaryFiles:
    foldername_search = re.search(r'[^\\/:*?"<>|\r\n]+$', h)
    foldername = (foldername_search.group()).split('.')[0]
    with open(h, 'r') as f:
        hypothesis = f.read()
    f.close()
    if foldername in gold_list:
        files = glob.glob('../data/raw/OpinosisDataset1.0_0/summaries-gold/'+foldername+'/*')
        for r in files:
            with open(r, 'r') as f:
                reference = f.read()
                rouge = Rouge()
                scores = rouge.get_scores(hypothesis, reference)[0]
                rouge_1.append(pd.DataFrame(scores['rouge-1'], index=[foldername]))
        frame = pd.concat(rouge_1)
    

In [66]:
summaryFiles = glob.glob(r'../data/processed/lsa/*')

rouge_1_lsa = []
rouge_2_lsa = []
rouge_l_lsa = []

for h in summaryFiles:
    foldername_search = re.search(r'[^\\/:*?"<>|\r\n]+$', h)
    foldername = (foldername_search.group()).split('.')[0]
    with open(h, 'r') as f:
        hypothesis = f.read()
    f.close()
    if foldername in gold_list:
        files = glob.glob('../data/raw/OpinosisDataset1.0_0/summaries-gold/'+foldername+'/*')
        for r in files:
            with open(r, 'r') as f:
                reference = f.read()
                rouge = Rouge()
                scores = rouge.get_scores(hypothesis, reference)[0]
                rouge_1.append(pd.DataFrame(scores['rouge-1'], index=[foldername]))
        frame = pd.concat(rouge_1)
        
           

In [67]:
frame

,f,p,r
accuracy_garmin_nuvi_255W_gps,0.152381,0.096386,0.363636
accuracy_garmin_nuvi_255W_gps,0.122449,0.072289,0.400000
accuracy_garmin_nuvi_255W_gps,0.108696,0.060241,0.555556
accuracy_garmin_nuvi_255W_gps,0.164948,0.096386,0.571429
accuracy_garmin_nuvi_255W_gps,0.104167,0.060241,0.384615
bathroom_bestwestern_hotel_sfo,0.138614,0.090909,0.291667
bathroom_bestwestern_hotel_sfo,0.065934,0.038961,0.214286
bathroom_bestwestern_hotel_sfo,0.129032,0.077922,0.375000
bathroom_bestwestern_hotel_sfo,0.081633,0.051948,0.190476
bathroom_bestwestern_hotel_sfo,0.086022,0.051948,0.250000
